<a href="https://colab.research.google.com/github/zegabr/pln-chatbot/blob/main/notebooks/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

In [4]:
intent_mapper = {
  'NEGATE': 0,
  'NEGATE_INTENT': 1,
  'REQUEST_ALTS': 2,
  'GOODBYE': 3,
  'REQUEST': 4,
  'THANK_YOU': 5,
  'AFFIRM': 6,
  'AFFIRM_INTENT': 7,
  'SELECT': 8,
  'INFORM': 9,
  'INFORM_INTENT': 10
}

In [5]:
TRAIN_DATASET_PATH = 'https://raw.githubusercontent.com/zegabr/pln-chatbot/main/dataset/classification/train_dataset.csv'
TEST_DATASET_PATH = 'https://raw.githubusercontent.com/zegabr/pln-chatbot/main/dataset/classification/test_dataset.csv'

In [6]:
from pandas import read_csv
train_restaurant_dataset = read_csv(TRAIN_DATASET_PATH)
# pega 2 np arrays, um com as frases e outro com os respectivos intents
train_dataset = train_restaurant_dataset.drop_duplicates(subset=['Phrase'])
train_phrases = np.array(train_restaurant_dataset.Phrase)[1:]
train_intents = np.array(train_restaurant_dataset.Intent.map(lambda x : intent_mapper[x]))[1:]
print(train_phrases)
print(train_intents)

['It has to be in San Fran.' "It'll be afternoon 12." "I'd like Palmer's."
 ... 'That sounds great! Thank you.' 'No, thank you very much!'
 'No, thank you very much!']
[9 9 9 ... 5 5 0]


In [33]:
test_dataset = read_csv(TEST_DATASET_PATH)
# pega 2 np arrays, um com as frases e outro com os respectivos intents
test_dataset = test_dataset.drop_duplicates(subset=['Phrase'])
test_phrases = np.array(test_dataset.Phrase)[1:]
test_intents = np.array(test_dataset.Intent.map(lambda x : intent_mapper[x]))[1:]

print(test_phrases[:10])
print(test_intents[:10])

517
517
["Check in the San Jose area. I'd like to find a place that serves Diner style food."
 'Great! can you make a reservation for one on Saturday this week?'
 'For lunch at 12 in the afternoon.'
 'On second thought, make it for Tuesday next week.'
 'Yes. Also see if they have outdoor seating and what their rating is.'
 'Great! Thank you.' "No, thank you. I'm good for now."
 'Can you help me find a place to eat?' "I'm interested in Napa."
 "Yes, I'm interested in Japanese."]
[9 8 9 0 4 5 5 9 9 9]


In [8]:
VOCAB_SIZE = 2000
encoder = tf.keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_phrases)

In [9]:
vocab = np.array(encoder.get_vocabulary())
print(len(encoder.get_vocabulary()))
vocab[:20]

1180


array(['', '[UNK]', 'i', 'for', 'the', 'a', 'to', 'you', 'that', 'is',
       'in', 'yes', 'make', 'it', 'at', 'reservation', 'me', 'restaurant',
       'no', 'thats'], dtype='<U15')

In [10]:
model = tf.keras.Sequential([
  encoder,
  tf.keras.layers.Embedding(
    input_dim=len(encoder.get_vocabulary()),
    output_dim=64,
    # Use masking to handle the variable sequence lengths
    mask_zero=True),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(11, activation='softmax')
])

In [11]:
# predict on a sample text without padding.

sample_text = ('Alright, fine. Now I would like to find a one-way flight to go '
               'there, and I do not mind which airline I will have.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[0.09102822 0.09212054 0.09073535 0.09071051 0.09199518 0.09116084
 0.09113285 0.08811726 0.09240778 0.08942764 0.09116379]


In [12]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[0.09102822 0.09212054 0.09073535 0.09071051 0.09199518 0.09116084
 0.09113286 0.08811726 0.09240778 0.08942764 0.09116379]


In [13]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [14]:
history = model.fit(train_phrases, train_intents, epochs=20)

Epoch 1/20
112/112 [==============================] - 10s 28ms/step - loss: 2.3370 - accuracy: 0.3673
Epoch 2/20
112/112 [==============================] - 3s 29ms/step - loss: 1.9199 - accuracy: 0.4099
Epoch 3/20
112/112 [==============================] - 3s 28ms/step - loss: 1.7154 - accuracy: 0.4203
Epoch 4/20
112/112 [==============================] - 3s 28ms/step - loss: 1.5628 - accuracy: 0.5028
Epoch 5/20
112/112 [==============================] - 3s 29ms/step - loss: 1.4097 - accuracy: 0.5870
Epoch 6/20
112/112 [==============================] - 3s 30ms/step - loss: 1.2892 - accuracy: 0.6364
Epoch 7/20
112/112 [==============================] - 3s 28ms/step - loss: 1.1938 - accuracy: 0.6540
Epoch 8/20
112/112 [==============================] - 3s 29ms/step - loss: 1.1091 - accuracy: 0.6633
Epoch 9/20
112/112 [==============================] - 3s 30ms/step - loss: 1.0308 - accuracy: 0.6684
Epoch 10/20
112/112 [==============================] - 3s 29ms/step - loss: 0.9660 - accur

In [15]:
sample_test = ('I\'m looking for a restaurant, can you help?')
sample_test_intent = 'INFORM'
predictions = model.predict(np.array([sample_test]))
for i in predictions:
  for idx, intent in enumerate(i):
    print('{:.2f}%'.format(intent*100), [i for i in intent_mapper.keys()][idx])
print('uhu?')

12.82% NEGATE
0.00% NEGATE_INTENT
4.48% REQUEST_ALTS
0.00% GOODBYE
0.04% REQUEST
0.06% THANK_YOU
0.10% AFFIRM
0.02% AFFIRM_INTENT
0.72% SELECT
81.74% INFORM
0.02% INFORM_INTENT
uhu?


In [16]:
sample_test = ('Also see if they have outdoor seating and what their rating is')
sample_test_intent = 'REQUEST'
predictions = model.predict(np.array([sample_test]))
for i in predictions:
  for idx, intent in enumerate(i):
    print('{:.2f}%'.format(intent*100), [i for i in intent_mapper.keys()][idx])
print('uhu?')

0.02% NEGATE
0.00% NEGATE_INTENT
0.04% REQUEST_ALTS
0.02% GOODBYE
80.11% REQUEST
0.02% THANK_YOU
19.36% AFFIRM
0.00% AFFIRM_INTENT
0.32% SELECT
0.10% INFORM
0.01% INFORM_INTENT
uhu?


In [37]:
from sklearn.metrics import classification_report

test_phrases = np.array(test_dataset.Phrase)[1:]
test_intents = np.array(test_dataset.Intent.map(lambda x : intent_mapper[x]))[1:]

predictions = model.predict(np.array(test_phrases))
y_pred = [] 
for pred in predictions:
  y_pred.append(pred.argmax())

print(classification_report(test_intents, y_pred))


              precision    recall  f1-score   support

           0       0.45      0.50      0.48        20
           2       0.83      0.38      0.53        13
           3       0.00      0.00      0.00         0
           4       0.92      0.70      0.79        82
           5       0.91      0.86      0.88        97
           6       0.59      0.71      0.64        65
           8       0.80      0.10      0.18        40
           9       0.79      0.99      0.88       200

    accuracy                           0.78       517
   macro avg       0.66      0.53      0.55       517
weighted avg       0.80      0.78      0.76       517



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
